In [1]:
import os
import random
import keras.utils
import numpy as np
from keras.layers import Input, Dense,Flatten, Concatenate
from keras.models import Model

keras.utils.set_random_seed(42)
np.random.seed(42)
random.seed(42)


inp = Input(shape=(320, 240, 2))
height = Input(shape=(1,))


resnet = keras.applications.ResNet50V2(
    include_top=False,
    weights=None,
    input_tensor=inp,
    input_shape=(320, 240, 2),
    pooling=max,
    classifier_activation="relu",
)

flat = Flatten()(resnet.output)
concat = Concatenate()([flat, height])
dense = Dense(128, activation="relu")(concat)

chest_out = Dense(1, activation='linear', name='chest_out')(dense)
waist_out = Dense(1, activation='linear', name='waist_out')(dense)
pelvis_out = Dense(1, activation='linear', name='pelvis_out')(dense)
bicep_out = Dense(1, activation='linear', name='bicep_out')(dense)
thigh_out = Dense(1, activation='linear', name='thigh_out')(dense)
shoulder_to_wrist_out = Dense(1, activation='linear', name='shoulder_to_wrist_out')(dense)
leg_out = Dense(1, activation='linear', name='leg_out')(dense)
calf_out = Dense(1, activation='linear', name='calf_out')(dense)
wrist_out = Dense(1, activation='linear', name='wrist_out')(dense)
shoulder_to_shoulder_out = Dense(1, activation='linear', name='shoulder_to_shoulder_out')(dense)


model = Model(
    inputs=[inp,height],
    outputs=[
        chest_out,
        waist_out,
        pelvis_out,
        bicep_out,
        thigh_out,
        shoulder_to_wrist_out,
        leg_out,
        calf_out,
        wrist_out,
        shoulder_to_shoulder_out,
    ],
    name='ResNet50V2'
)

###TRAINING VARIABLES

sample_count_surreact = 79999
sample_count_bodym = 6134
epoch_count = 100
batch_size = 15
validation_count = 20000
initial_learning_rate=.001
decay_steps= (sample_count_surreact + sample_count_bodym)/batch_size
decay_rate=0.97
model_name = "ResNetV2+Side+Height SR"


folder_path = f"{model_name} - SR{sample_count_surreact+sample_count_bodym}s{epoch_count}e{validation_count}v {"" if initial_learning_rate == 0.001 else initial_learning_rate + "ilr"}{decay_steps}s{decay_rate}d"

try:
    os.mkdir(os.path.join("./logs/" + folder_path))
except OSError as error:
    print(error)

2024-05-26 16:39:52.134464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 16:39:53.165304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from DatasetLoaderClassBCKP import DatasetLoader
quickTrain = {
    'image_location': ["Surreact-APose/train/", "BodyM/train/"],
    'data_location': ["Surreact-APose/train/measurements/", "BodyM/train/measurements/"],
    'samples': [sample_count_surreact, sample_count_bodym],
    'dataset': ["Surreact", "BodyM"],
    'batch_size': batch_size,
    'seed':  69,#np.random.randint(0, 10000), 
    'shuffle': True,
    'side':True,
    'height': True}

quickValidate = {
    'image_location': ["Surreact-APose/test/"],
    'data_location': ["Surreact-APose/test/measurements/"],
    'samples': [validation_count],
    'batch_size': batch_size,
    'dataset': ["Surreact"],
    'seed': 69, #np.random.randint(0, 10000),
    'side':True,
    'height': True}

train_generator = DatasetLoader(**quickTrain)
validation_generator = DatasetLoader(**quickValidate)

FileNotFoundError: [Errno 2] No such file or directory: 'Surreact-APose/train/measurements/Avatar_000000.npy'

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
from keras.losses import MeanSquaredError


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

stop_no_improvement = keras.callbacks.EarlyStopping(monitor="loss", patience=3)


model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss={
        'chest_out': MeanSquaredError(),
        'waist_out': MeanSquaredError(),
        'pelvis_out': MeanSquaredError(),
        'bicep_out': MeanSquaredError(),
        'thigh_out': MeanSquaredError(),
        'shoulder_to_wrist_out': MeanSquaredError(),
        'leg_out': MeanSquaredError(),
        'calf_out': MeanSquaredError(),
        'wrist_out': MeanSquaredError(),
        'shoulder_to_shoulder_out': MeanSquaredError(),
    },
    metrics={
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        'bicep_out': ['mae'],
        'thigh_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'leg_out': ['mae'],
        'calf_out': ['mae'],
        'wrist_out': ['mae'],
        'shoulder_to_shoulder_out': ['mae'],
    },
)



checkpoint_filepath = f'./models/{model_name}.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)

model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=[
        
        TensorBoard(write_graph=True,log_dir="./logs/"+folder_path),
        model_checkpoint_callback,
    ],
    batch_size=batch_size,
    epochs=epoch_count,
    verbose=1,
)

model.save(f"./models/fullSR/{model_name}.keras")
 